In [18]:
import numpy as np 
import tensorflow as tf
import keras
from keras import utils,callbacks
from keras.models import Sequential,Model
from keras.applications.vgg16 import VGG16
from keras.layers.core import Dropout,Flatten,Dense
from keras.layers.convolutional import Convolution2D,MaxPooling2D,ZeroPadding2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL.Image

In [2]:
train_dir=r"D:\jupyternotebooks\prototype 1\asl_alphabet_train\asl_alphabet_train"
test_dir=r"D:\jupyternotebooks\prototype 1\asl_alphabet_test\asl_alphabet_test"

In [3]:
classes=29
batch=256
epochs=1
lnr=0.0001

In [4]:
img_gen=ImageDataGenerator(rescale=1./255,validation_split=0.1)
train=img_gen.flow_from_directory(train_dir,target_size=(64,64),subset="training")
eval=img_gen.flow_from_directory(train_dir,target_size=(64,64),subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [5]:
adam=Adam(learning_rate=lnr)
model=Sequential()
model.add(VGG16(weights='imagenet',include_top=False,input_shape=(64,64,3)))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(29,activation='softmax'))

model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [6]:
earlystopping=callbacks.EarlyStopping(monitor="val_loss",
                                      mode='min',
                                      patience=5,
                                      restore_best_weights=True)

history=model.fit(train,
                  validation_data=eval,
                  epochs=epochs,
                  shuffle=True,
                  verbose=1,
                  callbacks=[earlystopping])

2447/2447 [==============================] - 1610s 655ms/step - loss: 0.4076 - accuracy: 0.8782 - val_loss: 0.2144 - val_accuracy: 0.9386


In [15]:
test=ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir,target_size=(64,64),class_mode=None)
pred=model.predict(test)
pred = np.argmax(pred, axis=1)
pred

Found 28 images belonging to 1 classes.


array([10, 12,  0,  3, 15,  6,  2,  8, 19, 22, 27, 17, 13, 18, 20,  1,  9,
       16, 25,  5, 21, 23, 24, 14,  4,  7, 11, 28], dtype=int64)

In [26]:
model.save('D:\jupyternotebooks\prototype 1')

INFO:tensorflow:Assets written to: D:\jupyternotebooks\prototype 1\assets


In [27]:
model = keras.models.load_model('D:\jupyternotebooks\prototype 1')
pred=model.predict(test)
pred = np.argmax(pred, axis=1)
pred

array([27, 21, 25, 16,  2,  8, 22,  4,  6, 13, 19, 14,  5,  3,  9, 12, 24,
        7, 20, 10, 15,  0,  1, 17, 23, 11, 28, 18], dtype=int64)